In [8]:
import pandas as pd
from e11 import H5Scan
from bokeh.io import output_notebook, show
from bokeh.models import Span,Label
from bokeh.plotting import figure
from scipy.optimize import curve_fit
import numpy as np
def load_data(year,month,day,file_nums):
    dfs =[]
    for f in file_nums:
        folder = 'data/' + year +month + day + "_" + f
        file = year +month +day + '_'+ f+'_scan.h5'
        path = folder + "/" + file
        scan = H5Scan(path) 
        dfs.append(scan.df('analysis'))
    return dfs
output_notebook()

year = '2019'
month = '08'
day='14'
file_nums = ['002','003']
data_files = load_data(year,month,day,file_nums)    
colors =['red','blue','black','grey','orange','cyan','brown','magenta']

Loading BokehJS ...

In [9]:
def damped_sine(t,A,f,phi,gamma):
    return A*np.sin(t*f+phi)*np.exp(-gamma*t)
def fit_damped_sine(data_file,freq_guess=1e-7):
    """
    given a data file will return the parameters of the fit
    """
    sine_max = np.max(data_file['f'])
    sine_min = np.min(data_file['f'])
    sine_ave = (sine_max + sine_min)/2

    amplitude_guess =  (sine_max - sine_min)/2
    phase_guess = 0.00
    decay_guess = 100  #1 micros
    frequency_guess= 0  #100 ns

    params = curve_fit(damped_sine, data_file['v0'],\
                       data_file['f'] - sine_ave,\
                       [amplitude_guess,frequency_guess,phase_guess,decay_guess])

    fitted_vals=[]
    t_max = np.max(data_file['v0'])
    ts = 1.1*np.linspace(0,t_max,1000)
    for t in ts:
        fitted_vals.append(damped_sine(t,*params[0]))
    fitted_vals = np.asarray(fitted_vals)
    
    return ts,fitted_vals + sine_ave,params[0]
def get_pi_time(A,f,phi,gamma):
    return ((np.pi/2) - phi)/f

def plot_pi(ts,fit,params):
    """
    will return coordinates to plot a line showing the pi pulse
    """
    zero_y = np.min(fit)
    max_y = np.max(fit)
    t_pi = get_pi_time(*params)
    return np.asarray([t_pi,t_pi]),np.asarray([zero_y,max_y])

In [10]:
p = figure(plot_width=800, plot_height=400)
for i,df in enumerate(data_files):
    p.line(df['v0']*1e9, df['f'], line_width=1,color=colors[i], legend='file {}'.format(file_nums[i]),line_alpha=0.2)
    ts,fit,params=fit_damped_sine(df)
    pi_time = get_pi_time(*params)
    xs,ys = plot_pi(ts,fit,params)

    p.line(ts*1e9,fit,color=colors[i],legend='pi time {} ns'.format(round(pi_time*1e9,3)))
    p.line(xs*1e9,ys,color=colors[i],legend='pi time {} ns'.format(round(pi_time*1e9,3)))

p.legend.click_policy = "hide"

show(p)